In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
print(os.getcwd())
os.chdir("/content/gdrive/My Drive/Colab Notebooks")
print(os.getcwd())

/content
/content/gdrive/My Drive/Colab Notebooks


In [3]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=d28597baaa6521e437800ff2fc8980137167f9a7fedd84d3847fe601eab4042b
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [4]:
import codecs
from collections import defaultdict
import csv
import string
from stop_words import get_stop_words    # download stop words package from https://pypi.org/project/stop-words/
import numpy as np


stop_words = get_stop_words('english')

admidic=defaultdict(list)
count=0


with open('NOTEEVENTS.csv', 'r') as csvfile:
     spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
     for row in spamreader:
         if row[6]=='Discharge summary':
             admidic[row[2]].append(row[-1].replace('\n',' ').translate(str.maketrans('','',string.punctuation)).lower())
             count=count+1

In [5]:
u=defaultdict(int)
for i in admidic:
    for jj in admidic[i]:
        line=jj.strip('\n').split()
        for j in line:
            u[j]=u[j]+1

In [6]:
u2=defaultdict(int)
for i in u:
        if i.isdigit()==False:
            if u[i]>10:
                if i not in stop_words:
                    u2[i]=u[i]  

In [16]:
length_u = len(u)
length_u2 = len(u2)

In [17]:
file1=codecs.open('DIAGNOSES_ICD.csv','r')
ad2c=defaultdict(list)
line=file1.readline()
line=file1.readline()

while line:
    line=line.strip().split(',')

    if line[4][1:-1]!='':
        ad2c[line[2]].append("d_"+line[4][1:-1])
    
    line=file1.readline()




codeu=defaultdict(int)
for i in ad2c:
    for j in ad2c[i]:
        codeu[j]=codeu[j]+1

In [18]:
cthre=0
fileo=codecs.open("combined_dataset",'w')

IDlist=np.load('IDlist.npy',encoding='bytes').astype(str)
for i in IDlist:
    if ad2c[i]!=[]:
        
        fileo.write('start! '+i+'\n')
        fileo.write('codes: ')
        tempc=[]
        for code in ad2c[i]:
            if codeu[code]>=cthre:
                if code[0:5] not in tempc:
                    tempc.append(code[0:5])
       
        for code in tempc:
            fileo.write(code+" ")
        fileo.write('\n')
        fileo.write('notes:\n')
        for line in admidic[i]:    
            thisline=line.strip('\n').split() 
            for j in thisline:
                if u2[j]!=0:
                    fileo.write(j+" ")
            fileo.write('\n')
        fileo.write('end!\n')
fileo.close()

In [19]:
average_words_admidic = sum([len(words) for notes in admidic.values() for words in notes])/count

In [22]:
average_words_admidic

8841.191041373299

In [24]:
len(admidic)

52726

In [25]:
length_u

529820

In [ ]:
length_u2

47965

In [ ]:
len(ad2c)

58933

In [ ]:
count

59652

total number of discharge notes = `count` = 59652

total number of aggregated discharge notes (per patient) = `len(admidic)` = 52726

number of unique words after preprocessing = `length_u2` = 47965

number of unique words before preprocessing = `length_u` = 529820

average number of words per note before preprocessing = 8841



